In [ ]:

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect("../cleandataset.sqlite")
df = pd.read_sql("SELECT * FROM player_attributes_dataset", conn)

y = df["overall_rating"].values.reshape(-1, 1)

features = [
    "acceleration", "sprint_speed", "stamina", "strength",
    "ball_control", "short_passing", "vision", "reactions",
    "positioning", "dribbling", "finishing", "long_passing",
    "shot_power", "balance", "agility"
]

X = df[features].values

X = (X - X.mean(axis=0)) / X.std(axis=0)

X = np.hstack([np.ones((X.shape[0], 1)), X])

def predict(X, w):
    return X @ w

def mse(y, y_hat):
    return np.mean((y - y_hat) ** 2)

def gradient_descent(X, y, lr=0.01, epochs=1000):
    w = np.zeros((X.shape[1], 1))
    n = len(y)

    for _ in range(epochs):
        y_hat = predict(X, w)
        grad = (2/n) * X.T @ (y_hat - y)
        w -= lr * grad

    return w

def metrics(y, y_hat):
    rmse = np.sqrt(np.mean((y - y_hat)**2))
    mae = np.mean(np.abs(y - y_hat))
    r2 = 1 - np.sum((y - y_hat)**2) / np.sum((y - y.mean())**2)
    return rmse, mae, r2

def k_fold_cv(X, y, k=5, lr=0.01, epochs=1000):
    idx = np.random.permutation(len(y))
    folds = np.array_split(idx, k)

    rmses, maes, r2s = [], [], []

    for i in range(k):
        test_idx = folds[i]
        train_idx = np.hstack([folds[j] for j in range(k) if j != i])

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        w = gradient_descent(X_train, y_train, lr, epochs)
        y_hat = predict(X_test, w)

        rmse, mae, r2 = metrics(y_test, y_hat)
        rmses.append(rmse)
        maes.append(mae)
        r2s.append(r2)

    return np.mean(rmses), np.mean(maes), np.mean(r2s)

learning_rates = [0.001, 0.01, 0.05]
epochs_list = [500, 1000, 2000]

results = []

for lr in learning_rates:
    for ep in epochs_list:
        rmse, mae, r2 = k_fold_cv(X, y, k=5, lr=lr, epochs=ep)
        results.append([lr, ep, rmse, mae, r2])
        print(f"lr={lr}, epochs={ep} -> RMSE={rmse:.2f}, R2={r2:.3f}")

results_df = pd.DataFrame(
    results,
    columns=["learning_rate", "epochs", "RMSE", "MAE", "R2"]
)

best = results_df.sort_values("RMSE").iloc[0]
print("\nMejor configuración:")
print(best)

w_final = gradient_descent(X, y, best.learning_rate, int(best.epochs))

coef = pd.DataFrame({
    "Variable": ["bias"] + features,
    "Coeficiente": w_final.flatten()
}).sort_values(by="Coeficiente", ascending=False)

coef.to_csv("resultados/coeficientes.csv", index=False)
results_df.to_csv("resultados/metricas.txt", index=False)

print("\nArchivos guardados en carpeta resultados.")


lr=0.001, epochs=500 -> RMSE=25.60, R2=-12.674
lr=0.001, epochs=1000 -> RMSE=10.13, R2=-1.142
lr=0.001, epochs=2000 -> RMSE=4.17, R2=0.637
lr=0.01, epochs=500 -> RMSE=3.95, R2=0.675
lr=0.01, epochs=1000 -> RMSE=3.93, R2=0.677
lr=0.01, epochs=2000 -> RMSE=3.93, R2=0.678
lr=0.05, epochs=500 -> RMSE=3.93, R2=0.678
lr=0.05, epochs=1000 -> RMSE=3.93, R2=0.678
lr=0.05, epochs=2000 -> RMSE=3.93, R2=0.678

Mejor configuración:
learning_rate       0.050000
epochs           2000.000000
RMSE                3.927578
MAE                 2.963096
R2                  0.678190
Name: 8, dtype: float64

Archivos guardados en carpeta resultados.
